In [2]:
import requests
import pandas as pd
import io
import time
from datetime import datetime
import locale
from tqdm import tqdm
import concurrent.futures
import os
import Funciones as fn
import ast

In [323]:
Lista = pd.read_csv('Juegos_19.11.txt', sep='\t', engine='python', header=None, encoding='utf-8-sig') #este codigo fue generado gracias a chat gpt, busca el poder volver el .txt a dataframe
Lista.columns = ['Ranking', 'vacio', 'name', 'jugadores_actuales', '24H-jugadores', 'Historico', 'nada2'] #acomodamos las columnas, hay 2 columnas cuyos datos son irrevelevantes o estan vacias, estas son las con nombres vacios y nada2
Lista['Ranking'] = Lista['Ranking'].astype(str).str.extract(r'(\d+)').astype(int) # Limpiar la columna Ranking (eliminar el punto si está como "1.")
Lista.head()

,Ranking,vacio,name,jugadores_actuales,24H-jugadores,Historico,nada2
0,1,NaN,Counter-Strike 2,"1,513,151","1,513,151","1,862,531",+
1,2,NaN,Dota 2,"743,785","775,263","1,295,114",+
2,3,NaN,PUBG: BATTLEGROUNDS,"625,767","653,641","3,257,248",+
3,4,NaN,Delta Force,"188,313","191,306","247,028",+
4,5,NaN,Apex Legends,"186,875","192,842","624,473",+


In [324]:
Lista.drop(columns=['vacio', 'nada2', 'Ranking', 'jugadores_actuales', '24H-jugadores'], inplace=True) #eliminamos las columnas que no vamos a usar
Lista.head()

,name,Historico
0,Counter-Strike 2,"1,862,531"
1,Dota 2,"1,295,114"
2,PUBG: BATTLEGROUNDS,"3,257,248"
3,Delta Force,"247,028"
4,Apex Legends,"624,473"


In [325]:
Lista.info() #vemos que la columna historico es de tipo str, cuando nos interesa que sea un int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       5000 non-null   object
 1   Historico  5000 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [326]:
Lista['Historico'] = Lista['Historico'].str.replace(',', '').astype(int) #eliminamos las comas y convertimos a int
Lista.info() #vemos que la columna historico ya es de tipo int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       5000 non-null   object
 1   Historico  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.3+ KB


In [327]:
print(Lista['name'].duplicated().value_counts()) #vemos los repetidos

name
False    4992
True        8
Name: count, dtype: int64


In [328]:
Lista.drop_duplicates(subset=['name'], inplace=True) #eliminamos los duplicados de la columna name

In [329]:
Lista.reset_index(drop=True, inplace=True) #reseteamos el indice

In [330]:
juegos = Lista['name'].tolist()

df_result = fn.obtener_appids_multithread(juegos,output_file='appids.csv', batch_size=50, workers=2, func_obtener_appid=fn.obtener_appid)

Reanudando desde archivo parcial con 4992 juegos ya guardados.
Juegos restantes por procesar: 0


0it [00:00, ?it/s]

✔ Todos los AppIDs obtenidos y guardados.


In [331]:
df_result.head()

,nombre_juego,appid
0,Counter-Strike 2,730.0
1,Dota 2,570.0
2,PUBG: BATTLEGROUNDS,578080.0
3,Delta Force,2507950.0
4,Apex Legends,1172470.0


In [332]:
df_result = df_result.rename(columns={'nombre_juego': 'name'})

In [333]:
df_result['appid'].isnull().value_counts() #vemos que hay 408 juegos cuyos appid son valores nulos

appid
False    4592
True      408
Name: count, dtype: int64

In [334]:
df_result.duplicated().value_counts() #vemos que hay 8 filas duplicadas, por lo que las eliminamos

False    4992
True        8
Name: count, dtype: int64

In [335]:
df_result.drop_duplicates(inplace=True)

In [336]:
df_filas_vacias = df_result[df_result['appid'].isna()] #existe la posibilidad de que estos juegos sean importantes pese a no haber conseguido su appid
print(df_filas_vacias)

                                  name  appid
13                Source SDK Base 2007    NaN
15           Grand Theft Auto V Legacy    NaN
88               Legends of Idleon MMO    NaN
116                  雀魂麻将(MahjongSoul)    NaN
142                      Rocket League    NaN
...                                ...    ...
4924           Universe Sandbox Legacy    NaN
4942               Winning Post 8 2017    NaN
4960                          Marfusha    NaN
4980         Shape of Dreams: Prologue    NaN
4998  BlazBlue: Chronophantasma Extend    NaN

[408 rows x 2 columns]


In [337]:
juegos_sin_appid = df_filas_vacias['name'].tolist()
historico_filtrado = Lista[Lista['name'].isin(juegos_sin_appid)]
historico_mas_100000 = historico_filtrado[historico_filtrado['Historico'] > 100000] #filtramos los juegos con mas de 100000 jugadores historicos

print(historico_mas_100000) #vemos algunos juegos importantes a considerar

                                                   name  Historico
13                                 Source SDK Base 2007     251292
15                            Grand Theft Auto V Legacy     364548
142                                       Rocket League     147632
157                                       RV There Yet?     100002
228                     Dying Light 2: Reloaded Edition     274983
315                                    EA SPORTS FC™ 24     107359
432                    Source SDK Base 2013 Multiplayer     184888
521                                        Dread Hunger     107982
640                                  EA SPORTS™ FIFA 23     110878
1036                                          Fall Guys     172213
1859  Call of Duty: Modern Warfare 2 (2009) - Multip...     112874
2073              Call of Duty: Black Ops - Multiplayer     118593
2126                     DELTARUNE (Chapter 1 & 2 DEMO)     102916
4540                                        MultiVersus     15

In [338]:
appid_dict = {
    'Rocket League': 252950,
    'Fall Guys': 1097150,
    'EA SPORTS FC™ 24': 2195250,
    'EA SPORTS™ FIFA 23': 1811260,
    'Dying Light 2: Reloaded Edition' : 534380  
}  #estos appid los asignamos manualmente
for juego, appid in appid_dict.items():
    df_result.loc[(df_result['name'] == juego) & (df_result['appid'].isna()), 'appid'] = appid
    
print(df_result[df_result['name'].isin(['Rocket League', 'Fall Guys','EA SPORTS FC™ 24','EA SPORTS™ FIFA 23', 'Dying Light 2: Reloaded Edition'])])

                                 name      appid
142                     Rocket League   252950.0
228   Dying Light 2: Reloaded Edition   534380.0
315                  EA SPORTS FC™ 24  2195250.0
640                EA SPORTS™ FIFA 23  1811260.0
1036                        Fall Guys  1097150.0


In [339]:
df_result = df_result.dropna() #ahora si eliminamos las filas con appid nulos y nombres nulos

In [340]:
df_result['appid'] = df_result['appid'].astype(int)
df_result.head(10) #ya tenemos el df con los appid

,name,appid
0,Counter-Strike 2,730
1,Dota 2,570
2,PUBG: BATTLEGROUNDS,578080
3,Delta Force,2507950
4,Apex Legends,1172470
5,ARC Raiders,1808500
6,Where Winds Meet,3564740
7,Battlefield™ 6,2807960
8,Wallpaper Engine,431960
9,Bongo Cat,3419430


In [341]:
Lista = Lista.merge(df_result, on='name', how='inner') #inner puesto que los juegos sin appid no los vamos a ocupar
Lista.head()

,name,Historico,appid
0,Counter-Strike 2,1862531,730
1,Dota 2,1295114,570
2,PUBG: BATTLEGROUNDS,3257248,578080
3,Delta Force,247028,2507950
4,Apex Legends,624473,1172470


In [342]:
Lista.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4589 entries, 0 to 4588
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       4589 non-null   object
 1   Historico  4589 non-null   int64 
 2   appid      4589 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 107.7+ KB


In [343]:
df_reviews = fn.procesar_reviews(Lista,output_file='Reviews.csv') #Vemos que hay algunas apis que dan errores de perdida de tiempo, luego veremos como arreglarlo

Reanudando desde archivo parcial con 4393 registros ya guardados.
Faltan por procesar: 0


0it [00:00, ?it/s]

✔ Proceso completado.


In [344]:
df_reviews.head()

,appid,review_score,review_score_desc,total_reviews
0,730,8.0,Very Positive,1378000.0
1,570,8.0,Very Positive,6095.0
2,578080,5.0,Mixed,278049.0
3,2507950,0.0,No user reviews,0.0
4,1172470,6.0,Mostly Positive,1032.0


In [345]:
df_reviews.duplicated().value_counts() 

False    4400
True      213
Name: count, dtype: int64

In [346]:
df_duplicados = df_reviews[df_reviews.duplicated(keep=False)]  #buscamos los duplicados para ver la razon de porque son duplicados
df_duplicados_ordenados = df_duplicados.sort_values(by='appid')
print(df_duplicados_ordenados)

        appid  review_score        review_score_desc  total_reviews
4007       20           8.0            Very Positive         4091.0
4276       20           8.0            Very Positive         4091.0
1122      550           9.0  Overwhelmingly Positive       219065.0
70        550           9.0  Overwhelmingly Positive       219065.0
1084      620           9.0  Overwhelmingly Positive       164054.0
...       ...           ...                      ...            ...
3230  3660220           5.0                    Mixed           96.0
507   3731020           7.0                 Positive           25.0
2107  3731020           7.0                 Positive           25.0
3711  3743240           7.0                 Positive           14.0
1353  3743240           7.0                 Positive           14.0

[403 rows x 4 columns]


In [347]:
Lista['appid'].duplicated().value_counts() #vemos que en lista hay datos que comparten el mismo appid, sin embargo esto es posible debido a que los juegos con expansiones o demos poseen la misma appidd

appid
False    4393
True      196
Name: count, dtype: int64

In [348]:
Lista['name'].duplicated().value_counts() #vemos que en lista no hay datos duplicados por nombre, sin embargo la cantidad de nombres y appid no coinciden

name
False    4589
Name: count, dtype: int64

In [349]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4613 entries, 0 to 4612
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   appid              4613 non-null   int64  
 1   review_score       4611 non-null   float64
 2   review_score_desc  4611 non-null   object 
 3   total_reviews      4611 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 144.3+ KB


In [350]:
df_reviews['review_score_desc'].unique() #vemos que hay scores que no tienen mucho sentido, asi que veremos que es lo que realmente son

array(['Very Positive', 'Mixed', 'No user reviews', 'Mostly Positive',
       'Overwhelmingly Positive', 'Mostly Negative', 'Positive',
       '4 user reviews', '1 user reviews', '8 user reviews',
       '5 user reviews', '3 user reviews', '2 user reviews',
       '9 user reviews', '7 user reviews', nan, '6 user reviews',
       'Negative', 'Overwhelmingly Negative'], dtype=object)

In [351]:
filas_9_reviews = df_reviews[df_reviews['review_score_desc'] == '9 user reviews']

print(filas_9_reviews)

        appid  review_score review_score_desc  total_reviews
512    404790           0.0    9 user reviews            9.0
1582  4028640           0.0    9 user reviews            9.0
1786  1182020           0.0    9 user reviews            9.0
2273  3543890           0.0    9 user reviews            9.0
2826  3351650           0.0    9 user reviews            9.0
2838  1461920           0.0    9 user reviews            9.0
3611  3261010           0.0    9 user reviews            9.0


In [352]:
filas_1_reviews = df_reviews[df_reviews['review_score_desc'] == '1 user reviews'] #realmente es super descriptivo el nombre de la columna xd

print(filas_1_reviews)

        appid  review_score review_score_desc  total_reviews
100   1818450           0.0    1 user reviews            1.0
672    705210           0.0    1 user reviews            1.0
924    312060           0.0    1 user reviews            1.0
1036  2167580           0.0    1 user reviews            1.0
1130   883860           0.0    1 user reviews            1.0
1367   979920           0.0    1 user reviews            1.0
1726  3117660           0.0    1 user reviews            1.0
1791  2730970           0.0    1 user reviews            1.0
1915  2056900           0.0    1 user reviews            1.0
1959  1144170           0.0    1 user reviews            1.0
2170  3761580           0.0    1 user reviews            1.0
2197  3330280           0.0    1 user reviews            1.0
2448  2376170           0.0    1 user reviews            1.0
2471  3916620           0.0    1 user reviews            1.0
2568   441960           0.0    1 user reviews            1.0
2624   860233           

In [353]:
df_reviews['review_score'].unique() #valores de 0 al 9 exceptuando el 2

array([ 8.,  5.,  0.,  6.,  9.,  4.,  7., nan,  3.,  1.])

In [354]:
df_reviews.dropna(inplace=True) #eliminamos filas con datos nulos

In [355]:
df_reviews = df_reviews.reset_index(drop=True)

In [356]:
#antes de usar merge, vamos a asegurarnos de que las columnas estan en los valores correctos
df_reviews['review_score'] = df_reviews['review_score'].astype(int)
df_reviews['total_reviews'] = df_reviews['total_reviews'].astype(int)
df_reviews['review_score_desc'] = df_reviews['review_score_desc'].astype('category')

In [357]:
Lista = Lista.merge(df_reviews, on='appid', how='inner') #inner porque los juegos sin reviews no nos interesan
Lista.head()

,name,Historico,appid,review_score,review_score_desc,total_reviews
0,Counter-Strike 2,1862531,730,8,Very Positive,1378000
1,Counter-Strike 2,1862531,730,8,Very Positive,1378020
2,Dota 2,1295114,570,8,Very Positive,6095
3,PUBG: BATTLEGROUNDS,3257248,578080,5,Mixed,278049
4,Delta Force,247028,2507950,0,No user reviews,0


In [358]:
Lista.drop_duplicates(inplace=True) #eliminamos los duplicados

In [359]:
df_info = fn.descargar_detalles_steam(Lista, output_file='Detalles_Steam.csv', batch_size=100, max_workers=4)

Reanudando desde archivo parcial con 4391 registros ya guardados.
Faltan por procesar: 0


0it [00:00, ?it/s]

✅ Proceso completado.


In [360]:
df_info.head()

,appid,name,type,required_age,is_free,detailed_description,price,developers,categories,release_date,genres
0,730,Counter-Strike 2,game,0.0,True,NaN,Free,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...","21 Aug, 2012","['Action', 'Free To Play']"
1,730,Counter-Strike 2,game,0.0,True,NaN,Free,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...","21 Aug, 2012","['Action', 'Free To Play']"
2,570,Dota 2,game,0.0,True,NaN,Free,Valve,"['Multi-player', 'Co-op', 'Steam Trading Cards...","9 Jul, 2013","['Action', 'Strategy', 'Free To Play']"
3,578080,PUBG: BATTLEGROUNDS,game,13.0,True,NaN,Free,PUBG Corporation,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","Dec 21, 2017","['Action', 'Adventure', 'Massively Multiplayer..."
4,2507950,Delta Force,game,0.0,True,NaN,Free,Team Jade,"['Single-player', 'Multi-player', 'MMO', 'PvP'...","4 Dec, 2024","['Action', 'Adventure', 'Massively Multiplayer..."


In [361]:
df_info = df_info.drop(columns=['detailed_description', 'type'])
df_info.head()

,appid,name,required_age,is_free,price,developers,categories,release_date,genres
0,730,Counter-Strike 2,0.0,True,Free,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...","21 Aug, 2012","['Action', 'Free To Play']"
1,730,Counter-Strike 2,0.0,True,Free,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...","21 Aug, 2012","['Action', 'Free To Play']"
2,570,Dota 2,0.0,True,Free,Valve,"['Multi-player', 'Co-op', 'Steam Trading Cards...","9 Jul, 2013","['Action', 'Strategy', 'Free To Play']"
3,578080,PUBG: BATTLEGROUNDS,13.0,True,Free,PUBG Corporation,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","Dec 21, 2017","['Action', 'Adventure', 'Massively Multiplayer..."
4,2507950,Delta Force,0.0,True,Free,Team Jade,"['Single-player', 'Multi-player', 'MMO', 'PvP'...","4 Dec, 2024","['Action', 'Adventure', 'Massively Multiplayer..."


In [362]:
df_info.info() #vemos las categorias de las columnas para transformarlas si es necesario

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4522 entries, 0 to 4521
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   appid         4522 non-null   int64 
 1   name          1443 non-null   object
 2   required_age  1443 non-null   object
 3   is_free       1443 non-null   object
 4   price         1443 non-null   object
 5   developers    1443 non-null   object
 6   categories    1432 non-null   object
 7   release_date  1441 non-null   object
 8   genres        1427 non-null   object
dtypes: int64(1), object(8)
memory usage: 318.1+ KB


In [363]:
df_info['required_age'].unique()

array(['0.0', '13.0', '17.0', '18.0', '16.0', nan, '0', '17', '18', '17+',
       '16', '13', '12', '14', '15', '10'], dtype=object)

In [364]:
df_info.isnull().value_counts()  #vemos la cantidad de valores nulos por columna

appid  name   required_age  is_free  price  developers  categories  release_date  genres
False  True   True          True     True   True        True        True          True      3079
       False  False         False    False  False       False       False         False     1415
                                                                                  True        15
                                                        True        False         False       10
                                                        False       True          False        2
                                                        True        False         True         1
Name: count, dtype: int64

In [365]:
df_info.duplicated().value_counts()

False    4454
True       68
Name: count, dtype: int64

In [366]:
df_info.drop_duplicates(inplace=True)

In [367]:
df_info['required_age'].fillna('0', inplace=True) #reemplazamos los valores nulos de required_age por 0

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22684\3612540402.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_info['required_age'].fillna('0', inplace=True) #reemplazamos los valores nulos de required_age por 0


In [368]:
Nulos = df_info[df_info['developers'].isnull()]

print(Nulos) #vemos una cantidad de 3031 juegos que son completamente Nulos, pero no nos interesa conseguir los juegos que no sean tan influentes, por que armaremos una brecha para juegos que tengan una cantidad minima de jugadores historicos

        appid name required_age is_free price developers categories  \
248    933110  NaN            0     NaN   NaN        NaN        NaN   
249   1891700  NaN            0     NaN   NaN        NaN        NaN   
250   1066780  NaN            0     NaN   NaN        NaN        NaN   
252   1783560  NaN            0     NaN   NaN        NaN        NaN   
254   1868140  NaN            0     NaN   NaN        NaN        NaN   
...       ...  ...          ...     ...   ...        ...        ...   
4517   282900  NaN            0     NaN   NaN        NaN        NaN   
4518   310470  NaN            0     NaN   NaN        NaN        NaN   
4519   314410  NaN            0     NaN   NaN        NaN        NaN   
4520   345180  NaN            0     NaN   NaN        NaN        NaN   
4521  2065810  NaN            0     NaN   NaN        NaN        NaN   

     release_date genres  
248           NaN    NaN  
249           NaN    NaN  
250           NaN    NaN  
252           NaN    NaN  
254         

In [369]:
juegos_nan = Nulos['appid'].tolist()
historico_filtrado_api = Lista[Lista['appid'].isin(juegos_nan)]
historico_mas_15000_api = historico_filtrado_api[historico_filtrado_api['Historico'] > 15000] #filtramos los juegos con mas de 15000 jugadores historicos

print(historico_mas_15000_api)
print(len(historico_mas_15000_api))

                                              name  Historico    appid  \
196                                    Dying Light      45876  3008130   
271         Age of Empires III: Definitive Edition      20146   933110   
273                              Transport Fever 2      24308  1066780   
278                                 DAVE THE DIVER      98480  1868140   
279                                        Necesse      29440  1169040   
...                                            ...        ...      ...   
4902                              Glass Masquerade      29689   511470   
4933  Dragon's Dogma 2 Character Creator & Storage      17528  2674810   
4944                       Oddworld: Abe's Oddysee      45049    15700   
4948                           Dead Island Riptide      17960   383180   
5003                                   Alien Swarm      60467      630   

      review_score        review_score_desc  total_reviews  
196              8            Very Positive       

In [370]:
re_nulos = fn.descargar_detalles_steam(historico_mas_15000_api, output_file='Detalles_Steam_Nulos.csv', batch_size=10, max_workers=1) #incluso ejecutando este codigo, nos dan datos nulos, pero cabe recalcar que depende tanto de la region como de la privacidad del propio publisher

Reanudando desde archivo parcial con 309 registros ya guardados.
Faltan por procesar: 0


0it [00:00, ?it/s]

✅ Proceso completado.


In [371]:
re_nulos.isnull().value_counts()

appid  name   type   required_age  is_free  detailed_description  price  developers  categories  release_date  genres
False  False  False  False         False    True                  False  False       False       False         False     237
       True   True   True          True     True                  True   True        True        True          True       87
       False  False  False         False    True                  False  False       False       False         True        1
                                                                                                 True          False       1
                                                                                     True        False         True        1
Name: count, dtype: int64

ademas de los juegos nulos, hay 4 juegos importantes que voy a añadir, que por alguna razon que desconozco no lograron pasar el filtro

In [375]:
# Lista de AppIDs
appids = [3164500, 1808500, 1030300, 3527290]

# Crear una lista para almacenar los resultados
resultados = []

for appid in appids:
    fila = fn.fetch_app_full(appid)  # Llamamos a la función combinada
    resultados.append(fila)

# Convertir la lista de diccionarios en DataFrame
df_juegos = pd.DataFrame(resultados)
df_juegos.head()



,appid,review_score,review_score_desc,total_reviews,name,type,required_age,is_free,developers,price,categories,release_date,genres
0,3164500,9,Overwhelmingly Positive,160341,Schedule I,game,0,False,TVGS,CLP$ 10.500,"[Single-player, Multi-player, Co-op, Online Co...","24 Mar, 2025","[Action, Indie, Simulation, Strategy, Early Ac..."
1,1808500,8,Very Positive,69832,ARC Raiders,game,0,False,Embark Studios,"R$ 171,80","[Multi-player, PvP, Online PvP, Co-op, Online ...","30 Oct, 2025",[Action]
2,1030300,8,Very Positive,108285,Hollow Knight: Silksong,game,0,False,Team Cherry,HK$ 125.00,"[Single-player, Steam Achievements, Full contr...","4 Sep, 2025","[Action, Adventure, Indie]"
3,3527290,8,Very Positive,84934,PEAK,game,0,False,Team PEAK,$7.99,"[Single-player, Multi-player, Co-op, Online Co...","Jun 16, 2025","[Action, Adventure, Indie]"


In [ ]:

df_juegos.replace({'price': {'$7.99': 'CLP$ 4.200', 'R$ 171,80': 'CLP$ 31.011', 'HK$ 125.00': 'CLP$ 10.500'}}, inplace=True) #ATENCION CON ESTA PARTE, ESTOS CAMBIOS SON ESPECIFICOS DE LOS DATOS OBTENIDOS
df_juegos = df_juegos.drop(columns=['type']) #AL RE EJECUTAR EL CODIGO EXISTE LA POSIBILIDAD DE QUE LA MONEDA EN LA QUE SE RECIBE CAMBIE, POR LO QUE TIENEN QUE CAMBIAR LOS VALORES MANUALMENTE
df_juegos['is_free'] = df_juegos['is_free'].astype(bool)#PORFAVOR TENER EN CUENTA ESO
df_juegos.loc[~df_juegos['is_free'], 'price'] = (
    df_juegos.loc[~df_juegos['is_free'], 'price']
    .str.replace('CLP$', '', regex=False)  # quita 'CLP$'
    .str.replace('.', '', regex=False)     # quita puntos de miles
    .str.strip()                           # quita espacios
)
df_juegos['price'] = df_juegos['price'].astype(int)
df_juegos['required_age'] = df_juegos['required_age'].astype(int) 
df_juegos['release_date'] = df_juegos['release_date'].apply(fn.parse_fecha)
df_juegos.head()

,appid,review_score,review_score_desc,total_reviews,name,required_age,is_free,developers,price,categories,release_date,genres
0,3164500,9,Overwhelmingly Positive,160341,Schedule I,0,False,TVGS,10500,"[Single-player, Multi-player, Co-op, Online Co...",2025-03-24,"[Action, Indie, Simulation, Strategy, Early Ac..."
1,1808500,8,Very Positive,69832,ARC Raiders,0,False,Embark Studios,31011,"[Multi-player, PvP, Online PvP, Co-op, Online ...",2025-10-30,[Action]
2,1030300,8,Very Positive,108285,Hollow Knight: Silksong,0,False,Team Cherry,10500,"[Single-player, Steam Achievements, Full contr...",2025-09-04,"[Action, Adventure, Indie]"
3,3527290,8,Very Positive,84934,PEAK,0,False,Team PEAK,4200,"[Single-player, Multi-player, Co-op, Online Co...",2025-06-16,"[Action, Adventure, Indie]"


In [ ]:
re_nulos.dropna(inplace=True)
df_info.dropna(inplace=True)
df_info = pd.concat([df_info, re_nulos], ignore_index=True)

In [ ]:
df_info.head()

,appid,name,required_age,is_free,price,developers,categories,release_date,genres,type,detailed_description
0,730,Counter-Strike 2,0.0,True,Free,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...","21 Aug, 2012","['Action', 'Free To Play']",NaN,NaN
1,570,Dota 2,0.0,True,Free,Valve,"['Multi-player', 'Co-op', 'Steam Trading Cards...","9 Jul, 2013","['Action', 'Strategy', 'Free To Play']",NaN,NaN
2,578080,PUBG: BATTLEGROUNDS,13.0,True,Free,PUBG Corporation,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","Dec 21, 2017","['Action', 'Adventure', 'Massively Multiplayer...",NaN,NaN
3,2507950,Delta Force,0.0,True,Free,Team Jade,"['Single-player', 'Multi-player', 'MMO', 'PvP'...","4 Dec, 2024","['Action', 'Adventure', 'Massively Multiplayer...",NaN,NaN
4,1172470,Apex Legends™,0.0,True,Free,Respawn,"['Multi-player', 'PvP', 'Online PvP', 'Co-op',...","4 Nov, 2020","['Action', 'Adventure', 'Free To Play']",NaN,NaN


In [ ]:
df_info = df_info.drop(columns=['detailed_description', 'type'])
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   appid         1395 non-null   int64 
 1   name          1395 non-null   object
 2   required_age  1395 non-null   object
 3   is_free       1395 non-null   object
 4   price         1395 non-null   object
 5   developers    1395 non-null   object
 6   categories    1395 non-null   object
 7   release_date  1395 non-null   object
 8   genres        1395 non-null   object
dtypes: int64(1), object(8)
memory usage: 98.2+ KB


In [ ]:
df_info['required_age'].unique()

array(['0.0', '13.0', '17.0', '18.0', '16.0', '0', '17', '18', '17+',
       '16', '13', '12', '14', '15', '10'], dtype=object)

In [ ]:
df_info['is_free'].unique()

array([True, False], dtype=object)

In [ ]:
df_info['is_free'] = df_info['is_free'].astype(bool)

In [ ]:
df_info['price'].unique()

array(['Free', '746.000₫', 'CLP$ 62.900', 'CLP$ 3.000', '₹ 1,700',
       'CLP$ 12.360', '฿315.00', 'CLP$ 26.990', 'CLP$ 7.700', '$69.99',
       '59.95 SR', 'CLP$ 49.900', '฿239.00', 'CLP$ 5.750', 'R$ 14,87',
       'CDN$ 79.99', 'CLP$ 59.999', '2,49€', '59,99€', 'CLP$ 15.500',
       '150.00 SR', 'CLP$ 39.999', 'CLP$ 16.150', 'CLP$ 31.999', '$59.99',
       'R$ 179,00', 'CLP$ 33.000', 'CLP$ 10.900', '$44.99 USD',
       'CLP$ 35.200', '990.000₫', 'R$ 274,50', 'CLP$ 59.900',
       'CLP$ 29.900', 'CDN$ 31.99', 'CLP$ 24.999', 'CLP$ 21.500',
       'CLP$ 18.000', 'CLP$ 21.900', 'CLP$ 12.000', 'CLP$ 8.300',
       '฿400.00', 'CLP$ 16.000', '22.49 SR', 'CLP$ 5.900', '289,00zł',
       'CLP$ 10.500', 'CDN$ 31.19', '159.00 SR', 'CDN$ 60.00',
       'CLP$ 6.400', '69,99€', 'CLP$ 10.999', 'CLP$ 69.990', 'R$ 46,99',
       'CLP$ 17.990', '51.49 SR', 'CLP$ 1.236', 'CLP$ 28.000',
       'CLP$ 2.750', 'CLP$ 13.500', '59.00 SR', '74.95 SR', 'CLP$ 62.999',
       'Rp 90 999', 'CLP$ 19.900', '119.99

In [ ]:
df_info = df_info[df_info['price'].str.startswith('CLP$') | (df_info['price'] == 'Free')]
df_info.reset_index(drop=True, inplace=True)
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   appid         966 non-null    int64 
 1   name          966 non-null    object
 2   required_age  966 non-null    object
 3   is_free       966 non-null    bool  
 4   price         966 non-null    object
 5   developers    966 non-null    object
 6   categories    966 non-null    object
 7   release_date  966 non-null    object
 8   genres        966 non-null    object
dtypes: bool(1), int64(1), object(7)
memory usage: 61.4+ KB


In [ ]:
df_info.loc[~df_info['is_free'], 'price'] = (
    df_info.loc[~df_info['is_free'], 'price']
    .str.replace('CLP$', '', regex=False)  # quita 'CLP$'
    .str.replace('.', '', regex=False)     # quita puntos de miles
    .str.strip()                           # quita espacios
)


In [ ]:
df_info['price'] = df_info['price'].replace('Free', 0)
df_info['price'] = df_info['price'].astype(int)

In [ ]:
df_info.replace({'required_age': {'17+': '17', '0.0': '0', '13.0': '13', '17.0': '17', '18.0': '18', '16.0': '16'}}, inplace=True)

In [ ]:
df_info['required_age'] = df_info['required_age'].astype(int)   

In [ ]:
df_info['release_date'] = df_info['release_date'].apply(fn.parse_fecha)

In [ ]:
df = df_info.merge(Lista, on='appid', how='inner')
df.head()


,appid,name_x,required_age,is_free,price,developers,categories,release_date,genres,name_y,Historico,review_score,review_score_desc,total_reviews
0,730,Counter-Strike 2,0,True,0,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",2012-08-21,"['Action', 'Free To Play']",Counter-Strike 2,1862531,8,Very Positive,1378000
1,730,Counter-Strike 2,0,True,0,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",2012-08-21,"['Action', 'Free To Play']",Counter-Strike 2,1862531,8,Very Positive,1378020
2,730,Counter-Strike 2,0,True,0,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",2012-08-21,"['Action', 'Free To Play']",Counter-Strike,319586,8,Very Positive,1378000
3,730,Counter-Strike 2,0,True,0,Valve,"['Multi-player', 'Cross-Platform Multiplayer',...",2012-08-21,"['Action', 'Free To Play']",Counter-Strike,319586,8,Very Positive,1378020
4,570,Dota 2,0,True,0,Valve,"['Multi-player', 'Co-op', 'Steam Trading Cards...",2013-07-09,"['Action', 'Strategy', 'Free To Play']",Dota 2,1295114,8,Very Positive,6095


In [ ]:
df.duplicated()
df.drop(columns=['name_x'], inplace=True)
df.rename(columns={'name_y': 'name'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   appid              1028 non-null   int64         
 1   required_age       1028 non-null   int64         
 2   is_free            1028 non-null   bool          
 3   price              1028 non-null   int64         
 4   developers         1028 non-null   object        
 5   categories         1028 non-null   object        
 6   release_date       1024 non-null   datetime64[ns]
 7   genres             1028 non-null   object        
 8   name               1028 non-null   object        
 9   Historico          1028 non-null   int64         
 10  review_score       1028 non-null   int64         
 11  review_score_desc  1028 non-null   category      
 12  total_reviews      1028 non-null   int64         
dtypes: bool(1), category(1), datetime64[ns](1), int64(6), object(4)

In [377]:
df = pd.concat([df, df_juegos], ignore_index=True)

In [378]:
df, mapa_categorico = fn.encode_list_column(df, 'categories')
print(mapa_categorico)
df.head()

{'Multi-player': 0, 'Cross-Platform Multiplayer': 1, 'Steam Trading Cards': 2, 'Steam Workshop': 3, 'In-App Purchases': 4, 'Adjustable Text Size': 5, 'Camera Comfort': 6, 'Color Alternatives': 7, 'Custom Volume Controls': 8, 'Playable without Timed Input': 9, 'Stereo Sound': 10, 'Surround Sound': 11, 'Valve Anti-Cheat enabled': 12, 'Stats': 13, 'Remote Play on Phone': 14, 'Remote Play on Tablet': 15, 'Remote Play on TV': 16, 'Steam Timeline': 17, 'Co-op': 18, 'SteamVR Collectibles': 19, 'PvP': 20, 'Online PvP': 21, 'Single-player': 22, 'MMO': 23, 'Online Co-op': 24, 'Steam Achievements': 25, 'Full controller support': 26, 'Chat Speech-to-text': 27, 'Chat Text-to-speech': 28, 'Adjustable Difficulty': 29, 'Keyboard Only Option': 30, 'Subtitle Options': 31, 'Steam Cloud': 32, 'HDR available': 33, 'Includes level editor': 34, 'Partial Controller Support': 35, 'Family Sharing': 36, 'Singleplayer': 37, 'Multiplayer': 38, 'OS-onafhankelijke multiplayer': 39, 'Aangepaste volumebesturing': 40, 

,appid,required_age,is_free,price,developers,categories,release_date,genres,name,Historico,review_score,review_score_desc,total_reviews,categories_numerico,genres_numerico,type
0,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike 2,1862531.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)",NaN
1,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike,319586.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)",NaN
2,570,0,True,0,Valve,"(Multi-player, Co-op, Steam Trading Cards, Ste...",2013-07-09 00:00:00,"(Action, Strategy, Free To Play)",Dota 2,1295114.0,8,Very Positive,6095,"[0, 18, 2, 3, 19, 4, 6, 7, 8, 12, 17]","(0, 2, 1)",NaN
3,578080,13,True,0,PUBG Corporation,"(Multi-player, PvP, Online PvP, Stats, Remote ...",2017-12-21 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",PUBG: BATTLEGROUNDS,3257248.0,5,Mixed,278049,"[0, 20, 21, 13, 14, 15]","(0, 3, 4, 1)",NaN
4,2507950,0,True,0,Team Jade,"(Single-player, Multi-player, MMO, PvP, Online...",2024-12-04 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",Delta Force,247028.0,0,No user reviews,0,"[22, 0, 23, 20, 21, 18, 24, 1, 25, 26, 2, 4]","(0, 3, 4, 1)",NaN


In [379]:
df.drop(columns=['type'], inplace=True)
df.head()

,appid,required_age,is_free,price,developers,categories,release_date,genres,name,Historico,review_score,review_score_desc,total_reviews,categories_numerico,genres_numerico
0,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike 2,1862531.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)"
1,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike,319586.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)"
2,570,0,True,0,Valve,"(Multi-player, Co-op, Steam Trading Cards, Ste...",2013-07-09 00:00:00,"(Action, Strategy, Free To Play)",Dota 2,1295114.0,8,Very Positive,6095,"[0, 18, 2, 3, 19, 4, 6, 7, 8, 12, 17]","(0, 2, 1)"
3,578080,13,True,0,PUBG Corporation,"(Multi-player, PvP, Online PvP, Stats, Remote ...",2017-12-21 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",PUBG: BATTLEGROUNDS,3257248.0,5,Mixed,278049,"[0, 20, 21, 13, 14, 15]","(0, 3, 4, 1)"
4,2507950,0,True,0,Team Jade,"(Single-player, Multi-player, MMO, PvP, Online...",2024-12-04 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",Delta Force,247028.0,0,No user reviews,0,"[22, 0, 23, 20, 21, 18, 24, 1, 25, 26, 2, 4]","(0, 3, 4, 1)"


In [380]:
df, mapa_categorico_genero = fn.encode_list_column(df, 'genres')
print(mapa_categorico_genero)
df.head()

{'Action': 0, 'Free To Play': 1, 'Strategy': 2, 'Adventure': 3, 'Massively Multiplayer': 4, 'RPG': 5, 'Casual': 6, 'Indie': 7, 'Animation & Modeling': 8, 'Design & Illustration': 9, 'Photo Editing': 10, 'Utilities': 11, 'Simulation': 12, 'Racing': 13, 'Sports': 14, 'Early Access': 15, 'Video Production': 16, 'Education': 17, 'Software Training': 18, 'Audio Production': 19, 'Aksi': 20, 'Game Development': 21, 'Web Publishing': 22, 'Violent': 23, 'Gore': 24}


,appid,required_age,is_free,price,developers,categories,release_date,genres,name,Historico,review_score,review_score_desc,total_reviews,categories_numerico,genres_numerico
0,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike 2,1862531.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1]"
1,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike,319586.0,8,Very Positive,1378000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1]"
2,570,0,True,0,Valve,"(Multi-player, Co-op, Steam Trading Cards, Ste...",2013-07-09 00:00:00,"(Action, Strategy, Free To Play)",Dota 2,1295114.0,8,Very Positive,6095,"[0, 18, 2, 3, 19, 4, 6, 7, 8, 12, 17]","[0, 2, 1]"
3,578080,13,True,0,PUBG Corporation,"(Multi-player, PvP, Online PvP, Stats, Remote ...",2017-12-21 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",PUBG: BATTLEGROUNDS,3257248.0,5,Mixed,278049,"[0, 20, 21, 13, 14, 15]","[0, 3, 4, 1]"
4,2507950,0,True,0,Team Jade,"(Single-player, Multi-player, MMO, PvP, Online...",2024-12-04 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",Delta Force,247028.0,0,No user reviews,0,"[22, 0, 23, 20, 21, 18, 24, 1, 25, 26, 2, 4]","[0, 3, 4, 1]"


In [381]:
df['genres'] = df['genres'].apply(tuple)
df['categories'] = df['categories'].apply(tuple)
df['categories_numerico'] = df['categories_numerico'].apply(tuple)
df['genres_numerico'] = df['genres_numerico'].apply(tuple)
df.head()


,appid,required_age,is_free,price,developers,categories,release_date,genres,name,Historico,review_score,review_score_desc,total_reviews,categories_numerico,genres_numerico
0,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike 2,1862531.0,8,Very Positive,1378000,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)"
1,730,0,True,0,Valve,"(Multi-player, Cross-Platform Multiplayer, Ste...",2012-08-21 00:00:00,"(Action, Free To Play)",Counter-Strike,319586.0,8,Very Positive,1378000,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","(0, 1)"
2,570,0,True,0,Valve,"(Multi-player, Co-op, Steam Trading Cards, Ste...",2013-07-09 00:00:00,"(Action, Strategy, Free To Play)",Dota 2,1295114.0,8,Very Positive,6095,"(0, 18, 2, 3, 19, 4, 6, 7, 8, 12, 17)","(0, 2, 1)"
3,578080,13,True,0,PUBG Corporation,"(Multi-player, PvP, Online PvP, Stats, Remote ...",2017-12-21 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",PUBG: BATTLEGROUNDS,3257248.0,5,Mixed,278049,"(0, 20, 21, 13, 14, 15)","(0, 3, 4, 1)"
4,2507950,0,True,0,Team Jade,"(Single-player, Multi-player, MMO, PvP, Online...",2024-12-04 00:00:00,"(Action, Adventure, Massively Multiplayer, Fre...",Delta Force,247028.0,0,No user reviews,0,"(22, 0, 23, 20, 21, 18, 24, 1, 25, 26, 2, 4)","(0, 3, 4, 1)"


In [382]:
df.drop_duplicates(subset=['name'], inplace=True)

In [383]:
df[df['release_date'].isna()] #revisamos los nulos en release_date, y vemos que son juegos que no han sido lanzados aun

,appid,required_age,is_free,price,developers,categories,release_date,genres,name,Historico,review_score,review_score_desc,total_reviews,categories_numerico,genres_numerico
13,3898830,0,False,0,Playroom,"(Singleplayer, Multiplayer, PvP, Online PvP, O...",NaT,"(Casual,)",Banana,917272.0,0,No user reviews,0,"(37, 38, 20, 21, 39, 40, 41, 42, 43, 44, 45, 4...","(6,)"
79,1422450,0,False,0,Valve,"(Multi-player,)",NaT,"(Action,)",Deadlock,171490.0,0,No user reviews,0,"(0,)","(0,)"
447,3784030,0,False,0,Doraccoon,"(Single-player, Steam Achievements, Full contr...",NaT,"(Strategy,)",Coin,1620.0,0,No user reviews,0,"(22, 25, 26, 36)","(2,)"
490,3509420,0,False,0,Madruga Works,"(Single-player, Steam Achievements, Steam Clou...",NaT,"(Indie, Strategy)",Planetbase,6670.0,0,No user reviews,0,"(22, 25, 32, 36)","(7, 2)"


In [ ]:
df.loc[df['name'] == "ARC Raiders", 'Historico'] = 481966 #añadimos los valores historicos que faltaban
df.loc[df['name'] == "Hollow Knight: Silksong", 'Historico'] = 587150
df.loc[df['name'] == "Schedule I", 'Historico'] = 459075
df.loc[df['name'] == "PEAK", 'Historico'] = 170759

In [387]:
df.info() #con esto ya terminamos la limpieza de datos, sin embargo, hay algunos juegos importantes que no lograron pasar algunos filtros anteriores, por lo que los agregaremos manualmente con fn.fetch_app_full a otro df

<class 'pandas.core.frame.DataFrame'>
Index: 1026 entries, 0 to 1025
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   appid                1026 non-null   int64  
 1   required_age         1026 non-null   int64  
 2   is_free              1026 non-null   bool   
 3   price                1026 non-null   object 
 4   developers           1026 non-null   object 
 5   categories           1026 non-null   object 
 6   release_date         1022 non-null   object 
 7   genres               1026 non-null   object 
 8   name                 1026 non-null   object 
 9   Historico            1026 non-null   float64
 10  review_score         1026 non-null   int64  
 11  review_score_desc    1026 non-null   object 
 12  total_reviews        1026 non-null   int64  
 13  categories_numerico  1026 non-null   object 
 14  genres_numerico      1026 non-null   object 
dtypes: bool(1), float64(1), int64(4), object(9)

In [391]:
df['Historico'] = df['Historico'].astype(int)  

In [394]:
mapa_categorico_df = pd.DataFrame(list(mapa_categorico.items()), columns=["clave", "genero"])
mapa_categorico_df.to_csv('mapa_categorico.csv', index=False)
mapa_categorico_df_genero = pd.DataFrame(list(mapa_categorico_genero.items()), columns=["clave", "genero"])
mapa_categorico_df_genero.to_csv('mapa_categorico_genero.csv', index=False)
df.to_csv('Datos.csv', index=False)